# Exploratory Data Analysis

### Banking Marketing Campaign

In [ ]:
#Libraries

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
import json
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import f_classif, SelectKBest
from pickle import dump
import warnings

# To supress warnings
warnings.filterwarnings("ignore", category=FutureWarning)

***STEP 1: PROBLEM STATEMENT & DATA COLLECTION***

The bank uses phone calls to reach out to customers and conduct marketing campaigns targeting long-term deposits. The goal is to optimize these contacts by targeting customers who are most likely to stick with the product.

**Project Goal:**
Create a classification model to predict whether or not a customer will take out a long-term deposit based on data provided by the bank.

**1.1 PROBLEM STATEMENT**

**Customer Profile:**

* Which age groups are most likely to take up a long-term deposit?
* Is there a correlation between the type of work (job) or level of education (education) and the decision to hire the deposit?
* Are customers with personal loans or mortgages less likely to stick to long-term deposits?

**Demographic and Marital Status Analysis:**

* Is there a difference in the adherence rate between married, single and other marital statuses?
* Do customers of certain age groups or marital status tend to adhere more to long-term products?

**Contact History and Adhesion Probability:**

* Which contact method has the highest success rate?
* Does the duration of contact influence the probability of conversion? Is there an optimal duration to maximize adherence?
* Is there a relationship between the number of contacts made in previous campaigns (campaign, previous) and the probability of adherence?

**Temporal Analysis:**

*  Does the month (month) or day of the week (day_of_week) of the contact affect the probability of adherence?
* Do success rates vary throughout the year or across days of the week?

**Economic Indicators and Customer Behavior:**

* How do macroeconomic variables such as the 3-month EURIBOR rate (euribor3m) or employment rate (emp.var.rate) influence customers' decision?
* Is there a relationship between the consumer confidence index (cons.conf.idx) and the willingness of customers to join the deposit?

**Previous Campaign Analysis:**

* How does the result of previous campaigns (poutcome) influence customers’ decisions?
* Are customers who were contacted in previous campaigns more likely to participate in this campaign?

**Customer Profile and Product Adherence:**

* Is there a common profile among customers who subscribe to long-term deposits? How do age, marital status, type of work and education level correlate with adherence?
* Is there a pattern in clients showing a greater propensity to accept long-term investment products?

**1.2. DATA COLLECTION**

In [12]:
pd.options.display.max_columns=None
df = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/logistic-regression-project-tutorial/main/bank-marketing-campaign-data.csv", sep=';')
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [13]:
df.to_csv("../data/raw/bank_data.csv", index=False)

***STEP 2: EXPLORATION & DATA CLEANING***

In [35]:
def check_data(df):
    print("Dataset dimensions:")
    print(df.shape)
    print("\nDataset informations:")
    print(df.info())
    print("\nMissing values ​​by column:")
    print(df.isna().sum()[df.isna().sum()>0])

check_data(df)

Dataset dimensions:
(41188, 21)

Dataset informations:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188

In [ ]:
# another option to obtain the total number of null values in each column:
# df.isnull().sum()

In [23]:
# Identificar variáveis numéricas
numerical_vars = df.select_dtypes(include=['float', 'int']).columns
num_numerical_vars = len(numerical_vars)  # Contar o número de variáveis numéricas

# Identificar variáveis categóricas
categorical_vars = df.select_dtypes(include=['object']).columns
num_categorical_vars = len(categorical_vars)  # Contar o número de variáveis categóricas

# Exibir os resultados
print(f"Numerical variables: {num_numerical_vars}")
print("Numerical variables:", list(numerical_vars))
print('\n')
print(f"Categorical variables: {num_categorical_vars}")
print("Categorical variables:", list(categorical_vars))



Numerical variables: 10
Numerical variables: ['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']


Categorical variables: 11
Categorical variables: ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome', 'y']


##### Statements

* This DataFame is composed by 41 188 rows and 21 columns (variables). 
* There are no variables with null values. 
* The data has:
    * **10 numerical variables** - 'age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m' and 'nr.employed';
    * **6 categorical variables** - 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome' and 'y'.

****Statistics:****

In [36]:
df.describe(include='all')

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
count,41188.00000,41188,41188,41188,41188,41188,41188,41188,41188,41188,41188.000000,41188.000000,41188.000000,41188.000000,41188,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188
unique,NaN,12,4,8,3,3,3,2,10,5,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,2
top,NaN,admin.,married,university.degree,no,yes,no,cellular,may,thu,NaN,NaN,NaN,NaN,nonexistent,NaN,NaN,NaN,NaN,NaN,no
freq,NaN,10422,24928,12168,32588,21576,33950,26144,13769,8623,NaN,NaN,NaN,NaN,35563,NaN,NaN,NaN,NaN,NaN,36548
mean,40.02406,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,258.285010,2.567593,962.475454,0.172963,NaN,0.081886,93.575664,-40.502600,3.621291,5167.035911,NaN
std,10.42125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,259.279249,2.770014,186.910907,0.494901,NaN,1.570960,0.578840,4.628198,1.734447,72.251528,NaN
min,17.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,0.000000,0.000000,NaN,-3.400000,92.201000,-50.800000,0.634000,4963.600000,NaN
25%,32.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.000000,1.000000,999.000000,0.000000,NaN,-1.800000,93.075000,-42.700000,1.344000,5099.100000,NaN
50%,38.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,180.000000,2.000000,999.000000,0.000000,NaN,1.100000,93.749000,-41.800000,4.857000,5191.000000,NaN
75%,47.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,319.000000,3.000000,999.000000,0.000000,NaN,1.400000,93.994000,-36.400000,4.961000,5228.100000,NaN


***2.2. ELIMINATE DUPLICATES***

In [37]:
# Verificar o número de duplicatas no DataFrame
num_duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {num_duplicates}")

# Se houver duplicatas, eliminá-las
if num_duplicates > 0:
    df = df.drop_duplicates()
    print("Duplicate rows have been removed.")
else:
    print("No duplicate rows found.")


Number of duplicate rows: 12
Duplicate rows have been removed.


***2.3. ELIMINATE IRRELEVANT INFORMATION***

* all variables will be kept.

In the early stages of developing a classification model to predict whether or not a customer will take out a long-term deposit, I chose to keep all variables because I do not yet know the impact of each variable on the model.

***STEP 3: ANALYSIS OF UNIVARIATE VARIABLES***

**3.1. ANALYSIS OF CATEGORICAL VARIABLES**

In [ ]:
# Your code here